# P02-02: Connectedness and Connected Components

*April 30 2020*

A key characteristic of a network is whether it is connected, i.e. whether all nodes are connected via a path. For disconnected networks, where this is not the case, we can compute so-called connected components, i.e. the maximally connected subgraphs. In this second unit we implement an algorithm to compute connected components, and apply it to empirical data sets.

In [1]:
import numpy as np
import pathpy as pp
import scipy as sp

import sqlite3

[04-30 20:09:34: DEBUG] pathpy version 3.0.0a1
[04-30 20:09:34: DEBUG] platform is win32
[04-30 20:09:34: DEBUG] pathpy runs in a vs code environment


Again, we start our exploration with a simple undirected and disconnected network that you already know from the lecture. This network has two connected components.

In [2]:
n_undirected = pp.Network(directed=False)
n_undirected.add_edge('a', 'b')
n_undirected.add_edge('b', 'c')
n_undirected.add_edge('a', 'c')
n_undirected.add_edge('d', 'f')
n_undirected.add_edge('d', 'g')
n_undirected.add_edge('d', 'e')
n_undirected.add_edge('e', 'f')
n_undirected.add_edge('f', 'g')
n_undirected.plot()

[04-30 20:09:34: DEBUG] Draw d3js object as html file
[04-30 20:09:34: DEBUG] Generate single html document.


We have seen that - for a suitable ordering of rows and columns - the connected components can be recognised as blocks in the powers of the adjacency matrix. In `pathpy` adjacency matrices are sparse by default, but we can output a dense adjacency matrix as follows:

In [3]:
m = n_undirected.adjacency_matrix().todense()
print(m)

[[0. 1. 1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 0. 1. 1.]
 [0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0.]]


To compute the square of the matrix, we can use `python`'s product operator, which will perform a matrix multiplication:

In [4]:
m*m

matrix([[2., 1., 1., 0., 0., 0., 0.],
        [1., 2., 1., 0., 0., 0., 0.],
        [1., 1., 2., 0., 0., 0., 0.],
        [0., 0., 0., 3., 2., 1., 1.],
        [0., 0., 0., 2., 3., 1., 1.],
        [0., 0., 0., 1., 1., 2., 2.],
        [0., 0., 0., 1., 1., 2., 2.]])

Higher powers of the matrix can be conveniently computed using the `np.linalg.matrix_power` (or via the ``**`` operator, which maps to the same function):

In [5]:
np.linalg.matrix_power(m, 10)

matrix([[ 342.,  341.,  341.,    0.,    0.,    0.,    0.],
        [ 341.,  342.,  341.,    0.,    0.,    0.,    0.],
        [ 341.,  341.,  342.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0., 3795., 3794., 2929., 2929.],
        [   0.,    0.,    0., 3794., 3795., 2929., 2929.],
        [   0.,    0.,    0., 2929., 2929., 2330., 2330.],
        [   0.,    0.,    0., 2929., 2929., 2330., 2330.]])

## Tarjan's Algorithm

The blocks in the powers of the adjacency matrix are visible because the rows/columsn of the adjacency matrix are ordered such that they correspond to the connected components. Nevertheless, we can use the idea that matrix algebra computes the number of walks in a network for the calculation of components based on the eigenvector of matrix representations (more on this in P09). Here we instead present the algorithm proposed by [Hopcroft and Tarjan 1973](https://dl.acm.org/citation.cfm?doid=362248.362272). It computes the connected components in a undirected networks as well as strongly connected components in directed networks. 

The algorithm is based on a modified depth-first search, which is executed recursively for (not previously discovered) nodes in each (strongly) connected component. The algorithm stores an incremental `dfs_num` of each visited node, which corresponds to the order in which nodes are discovered by the depth-first search. Each node is further assigned a `low_link` number, which stores the minimum `dfs_num` that can be reached from that node using at most one back edge, see https://en.wikipedia.org/wiki/Depth-first_search#Output_of_a_depth-first_search. When a node is first discovered, the `low_link` number is set to be equal to the `dfs_num`. The `low_link` number is then updated whenever we traverse a link to an already discovered node, assigning the minimum of the previous value of `low_link` and the `dfs_num` of the previously discovered node. The `dfs_num` and `low_link` numbers are used to terminate one depth-first exploration of a connected component and all nodes discovered until then are assigned to the same (strongly) connected component.

The algorithm considers each node and link exactly once. For a network with $n$ nodes and $m$ links its complexity is thus $\mathcal{O}(n+m)$.

In [6]:
def tarjans_algorithm(n, print_dfs_nums = False):
    # Tarjan's algorithm traverses a network with a depth-first-search
    
    # this number is incremented whenever a new node is discovered
    i = 1

    # stores the current value of i once a node is first discovered
    dfs_num = {}

    # as explained above
    low_link = {}

    # When the call to tarjan_visit finishes, the stack contains all nodes in the current connected component
    # Conveniently, python arrays can be used as stacks by calling the append() and pop() functions
    stack = []
    
    components = {}
    component_sizes = {}

    def dfs_visit(network: pp.Network, v: str):
        """Recursive method of Tarjan's algorithm that generates all nodes 
        that are in the same (strongly) connected component as node v."""
        nonlocal dfs_num
        nonlocal low_link
        nonlocal stack
        nonlocal i
        nonlocal components
        nonlocal component_sizes

        dfs_num[v] = i
        low_link[v] = i
        stack.append(v)
        i += 1

        # for all successors w of v, recursively call function
        # if w has not been previously discovered
        for w in network.successors[v]:

            # if w has not been previously discovered
            # recursively apply tarjan_visit to w
            if w.uid not in dfs_num:
                dfs_visit(network, w.uid)
                
                # update low_link[v] since we can reach w from v
                low_link[v] = min(low_link[v], low_link[w.uid])

            # we discovered a link to an already discovered node w
            elif w.uid in stack:
                
                # update low_link[v] since we can reach w from v
                low_link[v] = min(low_link[v], dfs_num[w.uid])
        
        # a "root" note completed a DFS traversal
        if low_link[v] == dfs_num[v]:
            # stack contains nodes that are in same 
            # strongly connected component as v
            components[v] = set()
            while True:
                x = stack.pop()
                components[v].add(x)
                if x == v:
                    break
            component_sizes[v] = len(components[v])
    
    for v in n.nodes.uids:
        # visit node if it has not been visited yet
        if v not in dfs_num:
            dfs_visit(n, v)
              
    if print_dfs_nums:
        print('node -> (dfsnum, low_link)')
        for v in n.nodes.uids:
            print('{0} -> ({1}, {2})'.format(v, dfs_num[v], low_link[v]))
    
    return components, component_sizes

In [7]:
components, sizes = tarjans_algorithm(n_undirected, print_dfs_nums=True)
print(components)
print(sizes)

node -> (dfsnum, low_link)
e -> (1, 1)
c -> (5, 5)
a -> (7, 5)
b -> (6, 5)
d -> (2, 1)
f -> (3, 1)
g -> (4, 2)
{'e': {'d', 'e', 'f', 'g'}, 'c': {'b', 'c', 'a'}}
{'e': 4, 'c': 3}


In the examplwe above, we find that the nodes `a`, `b`, `c` have the same `low_link` number. Nodes `e`, `e`, `f`, `g` have different values of `low_link` even though they are in the same component. In particualr, node `g` has a `low_link` of five, while all other nodes in that component have a value of four. This is because to reach node `e` (which has `dfs_num` 4) from node `g` we need to traverse two back edges. Apart from the connected components, the function above also returns the number of nodes in each component, which we can use to calculate the size of the largest component, relative to the network.

In [8]:
def lcc_size(network, normalise=True):
    components, sizes = tarjans_algorithm(network, print_dfs_nums=False)
    if normalise:
        return max(sizes.values())/network.number_of_nodes()
    else:
        return max(sizes.values())

In our undirected example network, the largest connected component contains a bit more than half of all nodes.

In [9]:
lcc_size(n_undirected)

0.5714285714285714

## Strongly connected components in directed networks

In directed networks, we distinguish between strongly and weakly connected networks. The following network is weakly but not strongly connected, because frmom the nodes `a` and `b` we can only reach `c` and `d` in one direction, but not in the opposite direction.

In [10]:
n_directed = pp.Network(directed=True)
n_directed.add_edge('a', 'b')
n_directed.add_edge('b', 'a')
n_directed.add_edge('a', 'c')
n_directed.add_edge('b', 'c')
n_directed.add_edge('c', 'd')
n_directed.add_edge('d', 'c')
n_directed.plot()

[04-30 20:09:35: DEBUG] Draw d3js object as html file
[04-30 20:09:35: DEBUG] Generate single html document.


The fact that this network is weakly but not strongly connected is captured by inifinite values in the lower triangle of the distance matrix, while no infinite values exist in the upper triangle:

In [11]:
pp.algorithms.shortest_paths.distance_matrix(n_directed)

array([[ 0.,  1.,  1.,  2.],
       [ 1.,  0.,  1.,  2.],
       [inf, inf,  0.,  1.],
       [inf, inf,  1.,  0.]])

We can use Tarjan's algorithm to identify the strongly connected components:

In [12]:
components, sizes = tarjans_algorithm(n_directed)
print(components)
print(sizes)

{'d': {'d', 'c'}, 'b': {'b', 'a'}}
{'d': 2, 'b': 2}


## Computing connected components in `pathpy`

The `find_connected_components` function in `pathpy` returns a dictionary of connected components:

In [13]:
pp.algorithms.components.find_connected_components(n_undirected)

[04-30 20:09:35: DEBUG] Computing connected components
component calculation: 100%|██████████| 7/7 [00:00<?, ?it/s]
[04-30 20:09:35: DEBUG] Mapping component sizes


{0: {'a', 'b', 'c'}, 1: {'d', 'e', 'f', 'g'}}

In [14]:
pp.algorithms.components.find_connected_components(n_directed)

[04-30 20:09:35: DEBUG] Computing connected components
component calculation: 100%|██████████| 4/4 [00:00<00:00, 3997.43it/s]
[04-30 20:09:35: DEBUG] Mapping component sizes


{0: {'c', 'd'}, 1: {'a', 'b'}}

We can use the function `largest_connected_component` to extract the largest connected component and return it as a new network object:

In [15]:
lcc = pp.algorithms.components.largest_connected_component(n_undirected)
lcc.plot()

[04-30 20:09:35: DEBUG] Computing connected components
[04-30 20:09:35: DEBUG] Computing connected components
component calculation: 100%|██████████| 7/7 [00:00<?, ?it/s]
[04-30 20:09:35: DEBUG] Mapping component sizes
[04-30 20:09:35: DEBUG] Copying network
[04-30 20:09:35: DEBUG] Removing nodes outside largest component
[04-30 20:09:35: DEBUG] Draw d3js object as html file
[04-30 20:09:35: DEBUG] Generate single html document.


To compute the size of the largest connected component in a network we can use a special function:

In [16]:
pp.algorithms.components.largest_component_size(n_undirected)

[04-30 20:09:36: DEBUG] Computing connected components
[04-30 20:09:36: DEBUG] Computing connected components
component calculation: 100%|██████████| 7/7 [00:00<00:00, 10356.31it/s]
[04-30 20:09:36: DEBUG] Mapping component sizes


4

In [17]:
lcc = pp.algorithms.components.largest_connected_component(n_directed)
lcc.plot()

[04-30 20:09:36: DEBUG] Computing connected components
[04-30 20:09:36: DEBUG] Computing connected components
component calculation: 100%|██████████| 4/4 [00:00<?, ?it/s]
[04-30 20:09:36: DEBUG] Mapping component sizes
[04-30 20:09:36: DEBUG] Copying network
[04-30 20:09:36: DEBUG] Removing nodes outside largest component
[04-30 20:09:36: DEBUG] Draw d3js object as html file
[04-30 20:09:36: DEBUG] Generate single html document.


## Connected component size in empirical networks

We finally apply connected component analysis to empirical networks. We use the same data sets that we used in the previous unit, and study whether (i) they are connected, (ii) how large the largest connected component is, and (iii) what are the shortest path lengths and the diameter in the largest connected component:

In [18]:
n_highschool = pp.io.read_sql('networks.db', sql='SELECT source, target FROM "highschool"', directed=False)
n_physicians = pp.io.read_sql('networks.db', sql='SELECT source, target FROM "physicians"', directed=False)
n_gentoo = pp.io.read_sql('networks.db', sql='SELECT source, target FROM "gentoo"', directed=True)
n_lotr = pp.io.read_sql('networks.db', sql='SELECT source, target FROM "lotr"', directed=False)

[04-30 20:09:36: DEBUG] Load sql file as pandas data frame.
[04-30 20:09:36: INFO ] No column v, searching for synonyms
[04-30 20:09:36: INFO ] Remapping column 'source' to 'v'
[04-30 20:09:36: INFO ] No column w, searching for synonyms
[04-30 20:09:36: DEBUG] Creating False network
[04-30 20:09:36: INFO ] Remapping column 'target' to 'w'
[04-30 20:09:36: DEBUG] Load sql file as pandas data frame.
[04-30 20:09:36: INFO ] No column v, searching for synonyms
[04-30 20:09:36: INFO ] Remapping column 'source' to 'v'
[04-30 20:09:36: INFO ] No column w, searching for synonyms
[04-30 20:09:36: DEBUG] Creating False network
[04-30 20:09:36: INFO ] Remapping column 'target' to 'w'
[04-30 20:09:37: DEBUG] Load sql file as pandas data frame.
[04-30 20:09:37: INFO ] No column v, searching for synonyms
[04-30 20:09:37: INFO ] Remapping column 'source' to 'v'
[04-30 20:09:37: INFO ] No column w, searching for synonyms
[04-30 20:09:37: DEBUG] Creating True network
[04-30 20:09:37: INFO ] Remapping c

In [19]:
print(pp.algorithms.components.largest_component_size(n_highschool)/n_highschool.number_of_nodes())

[04-30 20:09:39: DEBUG] Computing connected components
[04-30 20:09:39: DEBUG] Computing connected components
component calculation: 100%|██████████| 120/120 [00:00<00:00, 11421.36it/s]
[04-30 20:09:39: DEBUG] Mapping component sizes
1.0


In [20]:
print(pp.algorithms.components.largest_component_size(n_physicians)/n_physicians.number_of_nodes())

[04-30 20:09:39: DEBUG] Computing connected components
[04-30 20:09:39: DEBUG] Computing connected components
component calculation: 100%|██████████| 241/241 [00:00<00:00, 6019.62it/s]
[04-30 20:09:39: DEBUG] Mapping component sizes
0.4854771784232365


In [21]:
print(pp.algorithms.components.largest_component_size(n_gentoo)/n_gentoo.number_of_nodes())

[04-30 20:09:39: DEBUG] Computing connected components
[04-30 20:09:39: DEBUG] Computing connected components
component calculation: 100%|██████████| 403/403 [00:00<00:00, 3762.95it/s]
[04-30 20:09:39: DEBUG] Mapping component sizes
0.009925558312655087


In [22]:
lcc_physicians = pp.algorithms.components.largest_connected_component(n_physicians)
lcc_gentoo = pp.algorithms.components.largest_connected_component(n_gentoo)
lcc_lotr = pp.algorithms.components.largest_connected_component(n_lotr)

[04-30 20:09:39: DEBUG] Computing connected components
[04-30 20:09:39: DEBUG] Computing connected components
component calculation: 100%|██████████| 241/241 [00:00<00:00, 6173.97it/s]
[04-30 20:09:39: DEBUG] Mapping component sizes
[04-30 20:09:39: DEBUG] Copying network
[04-30 20:09:40: DEBUG] Removing nodes outside largest component
[04-30 20:09:40: DEBUG] Computing connected components
[04-30 20:09:40: DEBUG] Computing connected components
component calculation: 100%|██████████| 403/403 [00:00<00:00, 3834.62it/s]
[04-30 20:09:40: DEBUG] Mapping component sizes
[04-30 20:09:40: DEBUG] Copying network
[04-30 20:09:40: DEBUG] Removing nodes outside largest component
[04-30 20:09:40: DEBUG] Computing connected components
[04-30 20:09:40: DEBUG] Computing connected components
component calculation: 100%|██████████| 139/139 [00:00<00:00, 10682.50it/s]
[04-30 20:09:40: DEBUG] Mapping component sizes
[04-30 20:09:40: DEBUG] Copying network
[04-30 20:09:40: DEBUG] Removing nodes outside lar

In [23]:
print("Diameter of physicians is ", pp.algorithms.shortest_paths.diameter(lcc_physicians))
print("Diameter of gentoo is ", pp.algorithms.shortest_paths.diameter(lcc_gentoo))
print("Diameter of lotr is ", pp.algorithms.shortest_paths.diameter(lcc_lotr))

print("Avg. path length of physicians is ", pp.algorithms.shortest_paths.avg_path_length(lcc_physicians))
print("Avg. path length of gentoo is ", pp.algorithms.shortest_paths.avg_path_length(lcc_gentoo))
print("Avg. path length of lotr is ", pp.algorithms.shortest_paths.avg_path_length(lcc_lotr))

Diameter of physicians is  5.0
Diameter of gentoo is  3.0
Diameter of lotr is  6.0
Avg. path length of physicians is  2.5649791803637956
Avg. path length of gentoo is  1.3125
Avg. path length of lotr is  2.652483849409668
